In [11]:
import pandas as pd
import re

data_df = pd.read_json('Trademarkia_Task/data/idmanual.json')

print(f"Dataset size: {data_df.shape}")
print(f"Total unique classes: {data_df['class_id'].nunique()}")
print(f"Total unique status: {data_df['status'].nunique()}")
print(f"Total unique status: {data_df['status'].unique()}")

Dataset size: (58702, 4)
Total unique classes: 49
Total unique status: 4
Total unique status: ['A' 'M' 'D' 'X']


In [10]:
data_df.head()

id_tx class_id                                        description status
0  009-4140      009  Bank note acceptors for separating good bank n...      A
1  009-4136      009                                Fingerprint imagers      A
2  009-4133      009          Laboratory swabs [laboratory instruments]      A
3  009-4131      009                               Ear plugs for divers      A
4  009-4130      009                                      DVD recorders      A

In [9]:
data_df.loc[data_df.class_id == "200"]

id_tx class_id                                        description  \
51718  200-2      200  Collective Membership Mark for indicating memb...   

      status  
51718      M

In [18]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [24]:
data_df["class_id"] = data_df["class_id"].astype(str)

In [25]:
# Split the data into features and labels
X = data_df['description']
y = data_df['class_id']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Convert text to TF-IDF features
    ('clf', LogisticRegression(solver="sag"))  # Use logistic regression as the classifier
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline on the testing set
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         000       1.00      0.40      0.57        10
         001       0.69      0.70      0.70       386
         002       0.81      0.68      0.74       114
         003       0.78      0.78      0.78       306
         004       0.77      0.61      0.68        98
         005       0.81      0.79      0.80       411
         006       0.91      0.86      0.89       340
         007       0.80      0.82      0.81       473
         008       0.85      0.68      0.76       205
         009       0.57      0.90      0.70      1107
         010       0.78      0.70      0.74       371
         011       0.74      0.72      0.73       334
         012       0.81      0.85      0.83       318
         013       0.86      0.65      0.74        83
         014       0.84      0.69      0.75       105
         015       0.92      0.58      0.72        84
         016       0.83      0.85      0.84       598
         017       0.75    

In [26]:
import joblib

# Save the trained model
joblib.dump(pipeline, 'product_classifier.pkl')

['product_classifier.pkl']

In [32]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Convert text to TF-IDF features
    ('clf', XGBClassifier(tree_method = "hist", objective='multi:softmax'))  # Use XGBoost as the classifier
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline on the testing set
y_pred = pipeline.predict(X_test)
y_pred_labels = label_encoder.inverse_transform(y_pred)  # Convert the encoded labels back to original labels
y_actual_test = label_encoder.inverse_transform(y_test)  # Convert the encoded labels back to original labels
print(classification_report(y_actual_test, y_pred_labels))

              precision    recall  f1-score   support

         000       1.00      0.90      0.95        10
         001       0.72      0.58      0.64       386
         002       0.73      0.58      0.65       114
         003       0.72      0.69      0.70       306
         004       0.70      0.59      0.64        98
         005       0.74      0.67      0.70       411
         006       0.89      0.85      0.87       340
         007       0.76      0.78      0.77       473
         008       0.87      0.68      0.77       205
         009       0.43      0.79      0.56      1107
         010       0.73      0.56      0.64       371
         011       0.64      0.66      0.65       334
         012       0.77      0.79      0.78       318
         013       0.78      0.48      0.60        83
         014       0.79      0.75      0.77       105
         015       0.83      0.58      0.69        84
         016       0.82      0.76      0.79       598
         017       0.64    